In [2]:
import os
os.chdir('..')

import os, sys
import argparse, datetime, gc, time, json
import numpy as np
import pandas as pd
from typing import Sequence
from src.dataloader import Dataloader
from multiprocessing import Pool, cpu_count
from sklearn.metrics import mean_squared_error as mse

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'MySQLdb'

In [2]:
df = pd.read_parquet("data/raw/raw_1605484653/experian_credit_pull_1605484653.parquet")

In [3]:
df.shape

(907714, 22)

In [4]:
df.columns

Index(['user_id', 'credit_pull_date', 'fico_score', 'vantage_score', 'all7120',
       'all8220', 'bcc2800', 'bcc7120', 'bcx3423', 'iln5520', 'iqt9415',
       'iqt9413', 'mtf5820', 'stu5031', 'credit_card_loan_amount',
       'delinquencies_90_days', 'education_loan_amount',
       'mortgage_loan_amount', 'secured_loan_amount',
       'total_outstanding_balance', 'total_tradelines_open',
       'unsecured_loan_amount'],
      dtype='object')

In [6]:
df.user_id.nunique()

831959

In [20]:
dl = Dataloader(debug=True)

In [21]:
dfs = dl.load_dataframes("joined")
df = dfs["joined"]

In [22]:
df.columns[df.columns.str.contains("first_transaction_datetime")]

Index([], dtype='object')

In [23]:
dfs = dl.load_dataframes("features")
df = dfs["features"]

In [25]:
df.columns

Index(['trnx_tjd', 'tseq', 'trnx_time', 'business_account_number',
       'trnx_created_dt', 'trnx_endbal', 'trnx_transaction_code',
       'trnx_transaction_comment', 'trnx_transaction_amount', 'trnx_is_credit',
       ...
       'trnx_sum_deposits_3d', 'trnx_sum_deposits_30d', 'trnx_deposits_ratio',
       'trnx_first_transaction_datetime', 'trnx_days_since_first_deposit',
       'trnx_nr_past_transactions', 'trnx_nr_transactions_per_day',
       'trnx_nr_transactions_30d',
       'trnx_nr_transactions_30d_div_nr_past_transactions',
       'trnx_nr_past_deposits'],
      dtype='object', length=144)

In [4]:
import json
params = json.load(open("config.json", "r"))

In [5]:
params

{'data': {'raw': {'banking': 'raw/raw_1605487999/banking_1605487999.parquet',
   'experian_credit_pull': 'raw/raw_1605487999/experian_credit_pull_1605487999.parquet',
   'giact': 'raw/raw_1605487999/giact_1605487999.parquet',
   'socure': 'raw/raw_1605487999/socure_1605487999.parquet',
   'transactions': 'raw/raw_1605487999/transactions_1605487999.parquet',
   'user_metadata_dw': 'raw/raw_1605487999/user_metadata_dw_1605487999.parquet',
   'banking_account_restrictions': 'raw/raw_1605487999/banking_account_restrictions_1605487999.parquet',
   'plaid': 'raw/raw_1605487999/plaid_1605487999.parquet',
   'quovo': 'raw/raw_1605487999/quovo_1605487999.parquet'},
  'processed': {'banking': 'processed/banking_1605490833.parquet',
   'experian_credit_pull': 'processed/experian_credit_pull_1605490833.parquet',
   'giact': 'processed/giact_1605490833.parquet',
   'socure': 'processed/socure_1605490833.parquet',
   'transactions': 'processed/transactions_1605490833.parquet',
   'user_metadata_dw':

In [6]:
params

{'data': {'raw': {'banking': 'raw/raw_1605487999/banking_1605487999.parquet',
   'experian_credit_pull': 'raw/raw_1605487999/experian_credit_pull_1605487999.parquet',
   'giact': 'raw/raw_1605487999/giact_1605487999.parquet',
   'socure': 'raw/raw_1605487999/socure_1605487999.parquet',
   'transactions': 'raw/raw_1605487999/transactions_1605487999.parquet',
   'user_metadata_dw': 'raw/raw_1605487999/user_metadata_dw_1605487999.parquet',
   'banking_account_restrictions': 'raw/raw_1605487999/banking_account_restrictions_1605487999.parquet',
   'plaid': 'raw/raw_1605487999/plaid_1605487999.parquet',
   'quovo': 'raw/raw_1605487999/quovo_1605487999.parquet'},
  'processed': {'banking': 'processed/banking_1605490833.parquet',
   'experian_credit_pull': 'processed/experian_credit_pull_1605490833.parquet',
   'giact': 'processed/giact_1605490833.parquet',
   'socure': 'processed/socure_1605490833.parquet',
   'transactions': 'processed/transactions_1605490833.parquet',
   'user_metadata_dw':

In [8]:
mc_constraints = {
    "vantage_score": -1,
    "first_deposit_amount": -1,
    "bcc7120": -1,
    "credit_card_loan_amount": -1,
    "plaid_max_avail_bal": -1,
    "total_tradelines_open": -1,
    "plaid_days_since_first_link": 1,
    "nr_transactions_30d_div_nr_past_transactions": 1,
    "plaid_min_avail_bal": -1,
    "nr_transactions_per_day": 1,
    "max_deposits_10d_div_mean_account_balance_30d": 1,
    "fico_score": -1,
    "sum_deposits_10d": -1,
    "quovo_largest_checking_or_savings_balance": -1,
    "total_outstanding_balance": -1,
    "rolling_trns_as_pct_of_bal": 1,
    "phone_risk_score": 1,
    "quovo_available_bal": -1,
    "email_risk_score": 1,
    "deposits_ratio": 1,
    "fraud_score_1": 1,
    "mean_deposits_10d_div_mean_deposits": -1,
    "fraud_score_2": 1,
    "nr_past_deposits": -1,
    "quovo_min_avail_bal": 1,
    "address_risk_score": 1,
}

params['monotone_constraints'] = [mc_constraints[f] for f in params["features"]]
params['mc_method'] = 'basic'

In [10]:
with open("config.json", "w") as f:
    json.dump(params, f)

In [1]:
import pandas as pd

In [3]:
cdf = pd.read_csv("../artifacts/ach-mc-refit/governance/valid_segmented_performance/customer_refit_2021Q1_pred_top_10_pct.csv")

In [8]:
cdf.round(2)

,Percentile,Threshold,False Positive Rate (%),Fraud Capture Rate (%),#Above Threshold,#Fraudulent Above Threshold,#Fraudulent Below Threshold,#Good Above Threshold,#True Negatives
0,99.0,0.92,0.31,20.26,2574,1797,7071,777,247683
1,98.0,0.85,0.99,30.32,5147,2689,6179,2458,246002
2,97.0,0.79,1.79,36.82,7720,3265,5603,4455,244005
3,96.0,0.74,2.66,41.61,10294,3690,5178,6604,241856
4,95.0,0.69,3.54,45.82,12867,4063,4805,8804,239656
5,94.0,0.64,4.44,49.58,15440,4397,4471,11043,237417
6,93.0,0.60,5.37,52.81,18013,4683,4185,13330,235130
7,92.0,0.56,6.28,56.34,20587,4996,3872,15591,232869
8,91.0,0.52,7.21,59.16,23160,5246,3622,17914,230546
9,90.0,0.48,8.15,61.73,25733,5474,3394,20259,228201


In [5]:
adf = pd.read_csv("../artifacts/ach-mc-refit/governance/valid_segmented_performance/deposit_v2_ach_dev_final_pred_top_10_pct.csv")

In [10]:
adf.round(2)

,Percentile,Threshold,False Positive Rate (%),Fraud Capture Rate (%),#Above Threshold,#Fraudulent Above Threshold,#Fraudulent Below Threshold,#Good Above Threshold,#True Negatives
0,99.0,0.94,0.28,21.29,2574,1888,6980,686,247774
1,98.0,0.90,0.93,31.86,5147,2825,6043,2322,246138
2,97.0,0.86,1.70,39.54,7720,3506,5362,4214,244246
3,96.0,0.83,2.51,45.74,10294,4056,4812,6238,242222
4,95.0,0.79,3.37,50.61,12867,4488,4380,8379,240081
5,94.0,0.75,4.25,54.97,15440,4875,3993,10565,237895
6,93.0,0.72,5.16,58.64,18013,5200,3668,12813,235647
7,92.0,0.68,6.08,61.93,20587,5492,3376,15095,233365
8,91.0,0.65,7.01,64.76,23160,5743,3125,17417,231043
9,90.0,0.62,7.97,66.94,25733,5936,2932,19797,228663
